In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader=WebBaseLoader(
    web_paths=("https://www.barcelonasc.com.ec/2025/07/01/comunicado-oficial-jean-carlos-montano/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("elementor-section elementor-inner-section elementor-element elementor-element-649b8c9e elementor-section-boxed elementor-section-height-default elementor-section-height-default")
)))

text_documents=loader.load()

In [30]:
print(text_documents[0].metadata['source'])
print(text_documents[0].page_content.strip())  # Print first 1000 characters of the content

https://www.barcelonasc.com.ec/2025/07/01/comunicado-oficial-jean-carlos-montano/
Barcelona Sporting Club informa que ha llegado a un acuerdo con el volante Jean Carlos Montaño, quien se incorpora a nuestra institución como agente libre.
La operación contempla un contrato por el plazo de un año, con inicio en el mes de julio de 2025.
El jugador, de 26 años de edad, ha superado satisfactoriamente todas las pruebas médicas y ya se encuentra a disposición del primer equipo, bajo las órdenes del director técnico Ismael Rescalvo.
Le damos la bienvenida a Jean Carlos y le deseamos muchos éxitos defendiendo nuestros colores.
Barcelona Sporting Club


In [31]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('CV_ES.pdf')
docs=loader.load()

In [43]:
print(f"Autor del documento: {docs[0].metadata['author']}")
print(f"Total de páginas: {docs[0].metadata['total_pages']}")

for page in docs:
    print("="*10)
    print(f"Página {page.metadata['page']}")
    print(f"{page.page_content[:100]}") 

Autor del documento: iesus davila
Total de páginas: 2
Página 0
Iesus Dávila Ingeniero Mecatrónico
Teléfono: +593 96 314 5136
Email: iesusdavila@gmail.com
Dirección
Página 1
INVESTIGACIÓN Software Architecture and Simulation Interface for Autonomous Underwater Vehicles
Pape


In [44]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)


In [46]:
print(f"Autor del documento: {documents[0].metadata['author']}")
print(f"Total de páginas: {documents[0].metadata['total_pages']}")

for page in documents:
    print("="*10)
    print(f"Página {page.metadata['page']}")
    print(f"{page.page_content[:100]}") 

Autor del documento: iesus davila
Total de páginas: 2
Página 0
Iesus Dávila Ingeniero Mecatrónico
Teléfono: +593 96 314 5136
Email: iesusdavila@gmail.com
Dirección
Página 0
Unilever
Ingeniero en Robótica e IA | Febrero 2025 - Presente
Digital Factory Intern | Agosto 2024 -
Página 1
INVESTIGACIÓN Software Architecture and Simulation Interface for Autonomous Underwater Vehicles
Pape


In [70]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.llamacpp import LlamaCppEmbeddings
import os

model_path = "/home/iesus_robot/projects/langchain-lab/models/models--mixedbread-ai--mxbai-embed-large-v1/snapshots/db9d1fe0f31addb4978201b2bf3e577f3f8900d2/gguf/mxbai-embed-large-v1-f16.gguf"
if not os.path.exists(model_path):
    print(f"Model path does not exist. Please check the path.")

llama = LlamaCppEmbeddings(
    model_path=model_path,
    n_ctx=2048,
    verbose=True,
    n_gpu_layers=10,  
    n_threads=8,
)
db = Chroma.from_documents(documents, llama)

llama_model_load_from_file_impl: using device CUDA0 (NVIDIA GeForce RTX 3050 Laptop GPU) - 2839 MiB free
llama_model_loader: loaded meta data with 23 key-value pairs and 389 tensors from /home/iesus_robot/projects/langchain-lab/models/models--mixedbread-ai--mxbai-embed-large-v1/snapshots/db9d1fe0f31addb4978201b2bf3e577f3f8900d2/gguf/mxbai-embed-large-v1-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = bert
llama_model_loader: - kv   1:                               general.name str              = mxbai-embed-large-v1
llama_model_loader: - kv   2:                           bert.block_count u32              = 24
llama_model_loader: - kv   3:                        bert.context_length u32              = 512
llama_model_loader: - kv   4:                      bert.embedding_length u32              = 1024
llama_model_

In [77]:
query = "Cual es el numero de telefono de Iesus?"
retireved_results=db.similarity_search(query)

for opts in retireved_results:
    print("="*20)  
    print(f"Page: {opts.metadata['page']}")
    print(f"Content: {opts.page_content[:100]}")  

llama_perf_context_print:        load time =     131.24 ms
llama_perf_context_print: prompt eval time =      48.94 ms /    17 tokens (    2.88 ms per token,   347.38 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      57.10 ms /    18 tokens


Page: 0
Content: Iesus Dávila Ingeniero Mecatrónico
Teléfono: +593 96 314 5136
Email: iesusdavila@gmail.com
Dirección
Page: 0
Content: Iesus Dávila Ingeniero Mecatrónico
Teléfono: +593 96 314 5136
Email: iesusdavila@gmail.com
Dirección
Page: 0
Content: Iesus Dávila Ingeniero Mecatrónico
Teléfono: +593 96 314 5136
Email: iesusdavila@gmail.com
Dirección
Page: 0
Content: Unilever
Ingeniero en Robótica e IA | Febrero 2025 - Presente
Digital Factory Intern | Agosto 2024 -


In [79]:
from langchain_community.vectorstores import FAISS

model_path = "/home/iesus_robot/projects/langchain-lab/models/models--mixedbread-ai--mxbai-embed-large-v1/snapshots/db9d1fe0f31addb4978201b2bf3e577f3f8900d2/gguf/mxbai-embed-large-v1-f16.gguf"
if not os.path.exists(model_path):
    print(f"Model path does not exist. Please check the path.")

db_faiss = FAISS.from_documents(documents, llama)

llama_perf_context_print:        load time =     131.24 ms
llama_perf_context_print: prompt eval time =     261.45 ms /   724 tokens (    0.36 ms per token,  2769.19 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     287.55 ms /   725 tokens


In [81]:
query = "Cual es el numero de telefono de Iesus?"
retireved_results=db_faiss.similarity_search(query)

for opts in retireved_results:
    print("="*20)  
    print(f"Page: {opts.metadata['page']}")
    print(f"Content: {opts.page_content[:100]}")  

llama_perf_context_print:        load time =     131.24 ms
llama_perf_context_print: prompt eval time =     185.02 ms /    17 tokens (   10.88 ms per token,    91.88 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     206.28 ms /    18 tokens


Page: 0
Content: Iesus Dávila Ingeniero Mecatrónico
Teléfono: +593 96 314 5136
Email: iesusdavila@gmail.com
Dirección
Page: 0
Content: Unilever
Ingeniero en Robótica e IA | Febrero 2025 - Presente
Digital Factory Intern | Agosto 2024 -
Page: 1
Content: INVESTIGACIÓN Software Architecture and Simulation Interface for Autonomous Underwater Vehicles
Pape
